In [1]:
import pandas as pd
import json 
import numpy as np
#import pyfolio as pf
from datetime import datetime, timedelta
#import seaborn as sns
#import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


Converte il file json in un dataframe con 
* index = data del trade
* Pair = la coppia su cui si fa trading
* Return = la percentuale di gain o loss

In [2]:
def obtain_dataframe(backtest_path):
    f = open(backtest_path)
    data = json.load(f) 
    #dataframe of trades
    df = pd.DataFrame.from_dict(data, orient='columns')
    df = df.rename(columns={0: "Pair", 1: "Return", 2: 'Date'})
    df = df.drop(df.columns[[3, 4,5,6,7,8,9]], axis=1)
    df['Date'] = pd.to_datetime(df['Date'], unit='s').dt.date
    df['Date'] =  df['Date'].astype('category')
    df['Pair'] =  df['Pair'].astype('category')
    df = df.set_index('Date')
    return df

Semplicemente calcola l'equity moltiplicando il trade per il capitale che si è utilizzato
ed esempio io uso un capitale di 30€ fissi per trade
per il momento non ragiono in btc

In [3]:
def normal_equity(df, capital):
    equity = (df['Return']*capital).cumsum()
    equity = equity + capital
    equity = equity.to_frame() #trasformo da series a dataframe 
    return equity

Funzione che serve per calcolare il twr della pair, pag 53 unger

In [30]:
# df -> dataframe of all trades
def twr(df):
    
    WCS = df['Return'].min() #Worst trade, it can be switch with strategy's stop loss 
    twr = [] 
    f = []
    
    for j in range(1,100):
        
        hpr = [] #initialize array every loop
        risk = j/100 # use j to calculate f
        f.append(risk)
           
        for i in range(len(df)):        
            
            p_n = df.iloc[i]['Return'] #nth trade       
            hpr.append(1 - f[j-1] * ( p_n / WCS))  # HRP = 1 - f x (P/WCS)
        
        twr.append(np.prod(hpr)) # TWR = HPR1 x HPR2 x ... HPRn
    
    #Found best f
    df_twr =pd.DataFrame({'TWR': twr, '%f': f})
    
    max_twr = df_twr['TWR'].max()
    best_f = df_twr.loc[df_twr['TWR'] == max_twr]
     
    return best_f

Con l'optimal f vado a fare trading sulla pair

In [35]:
def optimalf_equity(df,capital):
    
    risk = twr(df).iloc[0]['%f'] #estrae l'unico valore di optimal f
    
    equity = []
    equity_trade = [] #array con la dimensione monetaria del trade
    for i in range(len(df)):
        trade =  df.iloc[i] 
        importodarischiare = risk * capital
        equity_trade.append(importodarischiare)
        g_l = trade['Return'] * importodarischiare
        capital = capital + g_l
        equity.append(capital)

    equity = pd.DataFrame({'Return':equity , 'Date': df.index})
    equity = equity.set_index('Date') 
    return equity, risk , equity_trade

Completa con tutte le date il dataframe dei trade

In [14]:
def complete_date(df):
    start_date = df.index[0] 
    end_date = df.index[len(df)-1]
    delta = end_date - start_date
    dd = [start_date + timedelta(days=x) for x in range((end_date- start_date).days + 1)] 
    dff = pd.DataFrame()
    dff['Date'] = dd
    dff['Date'] = pd.to_datetime(dff['Date']).dt.date
    dff['Date'] = dff['Date'].astype('category')
    dff = dff.groupby(dff['Date']).sum()
    df_join = (dff.join(df))['Return'].fillna(0)
    df_join = df_join.to_frame()
    df_join.index = pd.to_datetime(df_join.index)
    return df_join

In [12]:
def plot_equities(optimal_equity,norma_equity):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pd.to_datetime(optimal_equity.index), y=optimal_equity['Return'],
                        mode='lines',
                        name='Optimal %f'))
    fig.add_trace(go.Scatter(x=pd.to_datetime(norma_equity.index), y=norma_equity['Return'],
                        mode='lines',
                        name='Normal'))
    fig.update_layout(title='Equity comparison',
                       xaxis_title='Date',
                       yaxis_title='Return [$]')
    
    return fig



In [13]:
def plot_capitaltrade(optimal_equity,norma_equity,capital):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=pd.to_datetime(optimal_equity.index) , y=equity_trade, name='Optimal %f'))
    fig.add_trace(go.Scatter(x=pd.to_datetime(norma_equity.index), y= [capital]*len(norma_equity), name='Normal'))
    fig.update_layout(title='Capital per trade',
                       xaxis_title='Date',
                       yaxis_title='$')
    
    return fig

# Sezione su cui giochicchiare

In [32]:
capital = 100 #capitale totale a tua disposizione per fare trading, importante aggingerlo per optimal f
#qui inserisci il path di dove si trova il tuo backtest
backtest_path = '/home/matteo/freqtrade/user_data/backtest_results/backtest-result.json' 
df = obtain_dataframe(backtest_path) 
df = complete_date(df)
norma_equity = normal_equity(df, capital) #dataframe per l'equity normale
[optimal_equity,f, equity_trade] = optimalf_equity(df,capital) # dataframe e f_optimal per l'equity con money man
plot_equities(optimal_equity,norma_equity)
#plot_capitaltrade(optimal_equity,norma_equity)
#se non funziona avvia la casella della funzione twr, non so perchè

In [33]:
plot_capitaltrade(optimal_equity,norma_equity,capital)

In [52]:
del df, optimal_equity, norma_equity

In [34]:
df


,Return
Date,
2020-01-05,-0.051678
2020-01-05,-0.046272
2020-01-06,0.099900
2020-01-07,-0.058104
2020-01-07,0.099900
...,...
2020-06-01,0.099900
2020-06-02,0.099900
2020-06-02,0.016119


In [36]:
backtest_path = '/home/matteo/freqtrade/user_data/backtest_results/backtest-result.json' 
df = obtain_dataframe(backtest_path) 
df = complete_date(df)
WCS = df['Return'].min() #Worst trade, it can be switch with strategy's stop loss 
twr = [] 
f = []
    
for j in range(1,100):
        
    hpr = [] #initialize array every loop
    risk = j/100 # use j to calculate f
    f.append(risk)
           
    for i in range(len(df)):        
            
        p_n = df.iloc[i]['Return'] #nth trade       
        hpr.append(1 - f[j-1] * ( p_n / WCS))  # HRP = 1 - f x (P/WCS)
        
    #print(hpr)    
    twr.append(np.prod(hpr)) # TWR = HPR1 x HPR2 x ... HPRn
    
#Found best f
df_twr =pd.DataFrame({'TWR': twr, '%f': f})
    
max_twr = df_twr['TWR'].max()
best_f = df_twr.loc[df_twr['TWR'] == max_twr]

figure = go.Figure()
figure.add_trace(go.Scatter(x=[round(num*100,1) for num in f], y=[num for num in twr],
                            mode='lines',
                            name='TWR'))

figure.update_layout(title='TWR distribution ',
                        xaxis_title='%f',
                        yaxis_title='TWR')


figure.show()

In [60]:
df[df['Return'] != 0]

,Return
Date,
2019-01-05,-0.001998
2019-01-05,0.099900
2019-01-05,-0.011799
2019-01-06,0.099900
2019-01-07,-0.093258
...,...
2020-05-28,0.025159
2020-05-29,-0.013676
2020-05-30,0.003411


# Qua sotto è codice a caso su cui faccio prove

In [60]:
#df = df.groupby(df['Date']).sum()
#dataframe preparato per kelly
df_kelly = df.copy()
df_kelly = df_kelly.rename(columns={0: "Pair", 1: "Return"})
df_kelly = df_kelly.drop(df_kelly.columns[[2,3, 4,5,6,7,8,9]], axis=1)
df_kelly['Pair'] =  df_kelly['Pair'].astype('category')

IndexError: index 2 is out of bounds for axis 0 with size 1

In [61]:
df

,Return
Date,
2019-01-17,0.171059
2019-01-18,0.000000
2019-01-19,0.000000
2019-01-20,0.000000
2019-01-21,-0.195971
...,...
2020-05-16,0.000000
2020-05-17,0.000000
2020-05-18,0.000000


rep = (df_kelly.groupby(df_kelly['Pair']).apply(kelly)).sort_values(ascending=True)

import plotly.express as px
fig = px.bar(y=rep, x=rep.index)
fig.show()